In [1]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

from nltk.tokenize import word_tokenize


class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
        

In [2]:
short_pos=open("/home/saloni/Machine-learning/positive.txt","r",encoding="ISO-8859-1").read()
short_neg=open("/home/saloni/Machine-learning/negative.txt","r",encoding="ISO-8859-1").read()


In [12]:
documents=[]
all_words=[]
allowed_word_types = ["J"]# we are taking only adjectives
# for r in short_pos.split('\n'):
#     documents.append((r,"pos"))
    
# for r in short_neg.split('\n'):
#     documents.append((r,"neg"))
    
# short_pos_words=word_tokenize(short_pos)
# short_neg_words=word_tokenize(short_neg)

# for w in short_pos_words:
#     all_words.append(w.lower())
    
# for w in short_neg_words:
#     all_words.append(w.lower())
    
for p in short_pos.split('\n'):
    documents.append((p,"pos"))
    words=word_tokenize(p)
    pos=nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

for p in short_neg.split('\n'):
    documents.append( (p, "neg") )
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())
            
save_documents = open("/home/saloni/Machine-learning/pickled_algos/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()
    
    
all_words=nltk.FreqDist(all_words)

word_features=list(all_words.keys())[:5000]
save_word_features = open("/home/saloni/Machine-learning/pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]

random.shuffle(featuresets)
training_set = featuresets[:10000]
testing_set =  featuresets[10000:]

In [21]:
f_sets=open("/home/saloni/Machine-learning/pickled_algos/featuresets.pickle","wb")
pickle.dump(featuresets,f_sets)
f_sets.close()

In [15]:
featuresets[1][1]

'neg'

In [13]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

save_classifier = open("pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

Original Naive Bayes Algo accuracy percent: 70.63253012048193
Most Informative Features
              engrossing = True              pos : neg    =     20.8 : 1.0
                mediocre = True              neg : pos    =     17.2 : 1.0
                 generic = True              neg : pos    =     16.5 : 1.0
                 routine = True              neg : pos    =     15.8 : 1.0
               inventive = True              pos : neg    =     14.9 : 1.0
                  boring = True              neg : pos    =     14.6 : 1.0
                    flat = True              neg : pos    =     14.3 : 1.0
              refreshing = True              pos : neg    =     13.5 : 1.0
               wonderful = True              pos : neg    =     12.9 : 1.0
                    warm = True              pos : neg    =     12.5 : 1.0
                    dull = True              neg : pos    =     11.8 : 1.0
               realistic = True              pos : neg    =     11.6 : 1.0
            

In [14]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)
save_classifier = open("pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

MNB_classifier accuracy percent: 70.48192771084338


In [15]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)
save_classifier = open("pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()


BernoulliNB_classifier accuracy percent: 70.18072289156626


In [16]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)
save_classifier = open("pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()

LogisticRegression_classifier accuracy percent: 71.3855421686747


In [17]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)



/home/saloni/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 69.7289156626506


NameError: name 'SGDC_classifier' is not defined

In [19]:
save_classifier = open("pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDClassifier_classifier, save_classifier)
save_classifier.close()

In [20]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)
save_classifier = open("pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

LinearSVC_classifier accuracy percent: 68.37349397590361


In [9]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

NuSVC_classifier accuracy percent: 75.15060240963855


In [10]:
voted_classifier = VoteClassifier(
                                  NuSVC_classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

voted_classifier accuracy percent: 74.84939759036145


In [22]:
import sentiment_mod as s

10664


In [23]:
print(s.sentiment("This movie was awesome! The acting was great, plot was wonderful, and there were pythons...so yea!"))
print(s.sentiment("This movie was utter junk. There were absolutely 0 pythons. I don't see what the point was at all. Horrible movie, 0/10"))

('pos', 1.0)
('neg', 1.0)
